## This is an example of how to combine a graph exploring widget with the JSON viewer when clicking on nodes.

In [ ]:
import json

from IPython.display import JSON, display

import ipycytoscape
import ipywidgets as W
import networkx as nx
import rdflib
from ipycytoscape import CytoscapeWidget, Edge, Node
from ipyradiant import (
    FileManager,
    GetOutgoingPredicateObjects,
    InteractiveViewer,
    PathLoader,
)
from ipyradiant.rdf2nx import RDF2NX, URItoID, URItoShortID
from rdflib.namespace import RDFS

## First, we will set up an InteractiveViewer instance using the basic Star Wars example used in other examples.

In [ ]:
lw = FileManager(loader=PathLoader(path="data"))
# here we hard set what we want the file to be, but ideally a user can choose a file to work with.
lw.loader.file_picker.value = lw.loader.file_picker.options["starwars.ttl"]
rdf_graph = lw.graph
qres = lw.graph.query(
    """
    PREFIX hum: <https://swapi.co/resource/human/>
    PREFIX film: <https://swapi.co/resource/film/>
    
    CONSTRUCT {
        ?s ?p ?o .
    }
    WHERE {
        ?s ?p ?o .
        
        VALUES (?s) {
            (hum:1)  # Luke
            (hum:4)  # Vader
            (film:1) # A New Hope
        }
    }
    """
)

simple_graph = rdflib.graph.Graph().parse(data=qres.serialize(format="xml"))
uri = RDFS.label
ns = {"rdfs": str(RDFS)}
initNs = {
    "rdf": "http://www.w3.org/1999/02/22-rdf-syntax-ns#",
    "rdfs": "http://www.w3.org/2000/01/rdf-schema#",
    "res": "https://swapi.co/resource/",
    "voc": "https://swapi.co/vocabulary/",
    "base": "https://swapi.co/resource/",
}

nx_graph = RDF2NX.convert(simple_graph, namespaces=initNs)
directed = ipycytoscape.CytoscapeWidget()
directed.graph.add_graph_from_networkx(nx_graph, multiple_edges=True, directed=True)
for node in directed.graph.nodes:
    # deal with inability to handle colons
    node.data["_label"] = node.data.get("rdfs:label", None)
    node.data["_attrs"] = json.dumps(node.data, indent=2)

# create InteractiveViewer instance
iv = InteractiveViewer(rdf_graph=rdf_graph, cyto_graph=directed)

## Then, we will need a class that is composed of both the interactive viewer and the `JSON` viewer. We will use these in an `ipywidgets.VBox`

In [ ]:
class CombinedExample(W.VBox):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.json_output = W.Output()
        self.cytoscape_graph = kwargs["cyto_graph"]
        self.children = [self.cytoscape_graph, self.json_output]
        self.cytoscape_graph.cyto_graph.on("node", "click", self.load_json)

    def load_json(self, node):
        data = node["data"]
        data.pop("_label", None)
        data.pop("_attrs", None)
        with self.json_output:
            self.json_output.clear_output()
            display(JSON(data))

In [ ]:
ce = CombinedExample(cyto_graph=iv)

## We can now see that if you click on a node in the cytoscape graph, the `JSON` details of that node will be displayed below the graph.

In [ ]:
ce